# Lab 4: Urban Heat Island Detection using Thermal Satellite Imagery

## Goal
In this exercise, you will:
1. Select and compare two thermal satellite images from Kraków (or another city that has recently undergone intensive, thoughtless concrete development) – one from **2013** and one from **2024**.
2. Each image must be acquired on a **hot summer day** (T > 27°C based on IMGW meteorological data).
3. Images must have **low cloud cover** (< 20%).
4. Visualize thermal data and analyze surface temperature differences (Urban Heat Island effect).

## Task 1: Area of Interest (AOI)
- Define an AOI over the Reduta Street area in Kraków.
- Use `ee.Geometry.Polygon` or `ee.Geometry.Rectangle`.

In [1]:
import ee

ee.Authenticate()
ee.Initialize(project='ee-martynadurda')

aoi = ee.Geometry.Rectangle([-3.70239, 40.80757, -3.52386, 40.04233])  # cords of madrid bb

print(aoi.getInfo())


Successfully saved authorization token.
{'type': 'Polygon', 'coordinates': [[[-3.70239, 40.04233], [-3.52386, 40.04233], [-3.52386, 40.80757], [-3.70239, 40.80757], [-3.70239, 40.04233]]]}


## Task 2: Download and Analyze IMGW Meteorological Data
- Visit: [IMGW Archive](https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/)
- Navigate to:
  - `dane_meteorologiczne/dobowe/synop/2013/`
  - `dane_meteorologiczne/dobowe/synop/2024/`
- Download `s_d_tmax.csv` for both years.
- Filter the rows for **station ID 12566 (e.g. Kraków-Balice)**.
- Identify days in **July or August** with **TMAX > 27°C**.

In [2]:
%pip install fsspec

Note: you may need to restart the kernel to use updated packages.


## Task 3: Select Landsat 8 Images Matching These Dates
- In Earth Engine, use `LANDSAT/LC08/C02/T1_L2` collection.
- Apply filters:
  - `.filterBounds(aoi)`
  - `.filterDate()` for the matching day
  - `.filterMetadata('CLOUD_COVER', 'less_than', 20)`
- Try to find **one image from 2013** and **one image from 2024** that match your hot days list.

In [5]:
import geemap 
start_2013 = ee.Date('2013-08-01') # dzien wczesniej bo na 08 nie ma zdj
end_2013 = ee.Date('2013-08-15')
start_2024 = ee.Date('2024-08-06') # zakres 26-31 bo i tak wszystko >27 stopni
end_2024 = ee.Date('2024-08-16')

image_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2').filterBounds(aoi).filterDate(start_2013, end_2013).filterMetadata('CLOUD_COVER', 'less_than', 20).first()
image_2024 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterBounds(aoi).filterDate(start_2024, end_2024).filterMetadata('CLOUD_COVER', 'less_than', 20).first()
image_2013 = image_2013.clip(aoi)
image_2024 = image_2024.clip(aoi)

print(image_2013.getInfo())
print(image_2024.getInfo())

vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 5000,
    'max': 15000,
    'gamma': 1.3
}
center = aoi.centroid(10).coordinates().reverse().getInfo()

Map = geemap.Map(center=center, zoom=15)

Map.addLayer(image_2013, vis_params, 'Landsat 2013-08-08')
Map.addLayer(image_2024, vis_params, 'Landsat 2024-08-25')

Map

{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [526, 2836], 'origin': [4169, 2182], 'crs': 'EPSG:32630', 'crs_transform': [30, 0, 314985, 0, -30, 4583115]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [526, 2836], 'origin': [4169, 2182], 'crs': 'EPSG:32630', 'crs_transform': [30, 0, 314985, 0, -30, 4583115]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [526, 2836], 'origin': [4169, 2182], 'crs': 'EPSG:32630', 'crs_transform': [30, 0, 314985, 0, -30, 4583115]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [526, 2836], 'origin': [4169, 2182], 'crs': 'EPSG:32630', 'crs_transform': [30, 0, 314985, 0, -30, 4583115]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 6553

Map(center=[40.42425881714212, -3.6131250000012645], controls=(WidgetControl(options=['position', 'transparent…

## Task 4: Process Thermal Band (ST_B10)
- Convert Band 10 to Brightness Temperature (Kelvin):
  `TB = ST_B10 * 0.00341802 + 149.0`
- Create a visualization of each image using the same color scale.

In [6]:
bt_2013 = image_2013.select('ST_B10').multiply(0.00341802).add(149.0)
bt_2024 = image_2024.select('ST_B10').multiply(0.00341802).add(149.0)


thermal_vis = {
    'min': 290,
    'max': 320,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}

center = aoi.centroid(10).coordinates().reverse().getInfo()
Map = geemap.Map(center=center, zoom=15)
Map.addLayer(bt_2013, thermal_vis, '2013 Brightness Temp')
Map.addLayer(bt_2024, thermal_vis, '2024 Brightness Temp')
Map

Map(center=[40.42425881714212, -3.6131250000012645], controls=(WidgetControl(options=['position', 'transparent…

## Task 5: Compare and Interpret
- Compare the two maps.
- Optionally calculate difference: `TB_2024 - TB_2013`
- Discuss: did the surface temperature increase in the area?
- Is there evidence of an Urban Heat Island effect related to development?

In [8]:
bt_diff = bt_2024.subtract(bt_2013)

diff_vis = {
    'min': -5,
    'max': 5,
    'palette': ['blue', 'white', 'red']  # Blue = cooler in 2024, Red = hotter in 2024
}



mean_2013 = bt_2013.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

mean_2024 = bt_2024.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

mean_diff = bt_diff.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).getInfo()

print("Average Brightness Temperature 2013 (K):", mean_2013)
print("Average Brightness Temperature 2024 (K):", mean_2024)
print("Difference (K):", mean_diff)


center = aoi.centroid(10).coordinates().reverse().getInfo()
Map = geemap.Map(center=center, zoom=15)


Map.addLayer(bt_diff, diff_vis, 'Temperature Difference (2024 - 2013)')


Map


Average Brightness Temperature 2013 (K): {'ST_B10': 316.5839673578515}
Average Brightness Temperature 2024 (K): {'ST_B10': 321.527638082652}
Difference (K): {'ST_B10': 4.943670724801522}


Map(center=[40.42425881714212, -3.6131250000012645], controls=(WidgetControl(options=['position', 'transparent…

The heat impact in Madrid is huge. This city is located i  the valley, really dry climate with extremely high temperatures. As one of the biggest cities in  Eu that is constantly developing we can see a big impact in the city itself and the cummune of Madrid.